### Import modules


In [24]:
import mysql.connector as connector
from mysql.connector import errors
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import errorcode
import datetime

### Creating a database config file


In [25]:
#database config file
dbconfig = {"database":"little_lemon_db","user":"root","password":"root@123"}
try:
    pool = MySQLConnectionPool(pool_name='pool_b',pool_size=2,**dbconfig)
    print("The pool is created with the name: ",pool.pool_name)
    print("The pool size is: ",pool.pool_size)
except errors.Error as e:
    print("Error No: ",e.errno)
    print("Error Msg: ",e.msg)

The pool is created with the name:  pool_b
The pool size is:  2


In [30]:
#Ask user for values in bookings table
def prompt_bookings_insert():
    table_no = input('Enter Table No: ')
    first_name = input('Enter First Name: ')
    last_name = input('Enter Last Name:')
    booking_time = input('Enter Booking Time (HH:MM:SS): ')
    employee_id = input('Enter employee id: ')
    return (
        table_no,
        first_name,
        last_name,
        booking_time,
        employee_id
    )


In [31]:
#insert guest information in bookings table
guest_add = """
INSERT INTO Bookings(Bookings.TableNo,Bookings.GuestFirstName,Bookings.GuestLastName,Bookings.BookingSlot,Bookings.EmployeeID) 
VALUES (%s,%s,%s,%s,%s);
"""

In [33]:
#create guest list
guest_list = ["Guest 1","Guest 2","Guest 3","Guest 4"]
#get the connection from pool_b
for guest in guest_list:
    try:
        guest_connection = pool.get_connection()
        print(f"{guest} is connected")
    except errors.PoolError as e:
        print(e)
        print('Trying to get new connection... wait....')
        connection = connector.connect(user="root",password="root@123")
        guest_connection = pool.add_connection(cnx=connection)
        print("A new connection is added in the pool")
        new_guest_user_connection = pool.get_connection()
        print(f"{guest} is connected")
    finally:
        records = prompt_bookings_insert()
        if guest_connection.is_connected():
            cursor = guest_connection.cursor()
            try:
                cursor.execute(guest_add,records)
                guest_connection.commit()
            except errors.Error as e:
                print(e)
            finally:
                cursor.close()
            guest_connection.close()
        elif new_guest_user_connection.is_connected():
            cursor = new_guest_user_connection.cursor()
            try:
                cursor.execute(guest_add,records)
                new_guest_user_connection.commit()
            except errors.Error as e:
                print(e)
            finally:
                cursor.close()
            new_guest_user_connection.close()
        print("Connection is closed")

Guest 1 is connected
Connection is closed
Guest 2 is connected
Connection is closed
Guest 3 is connected
Connection is closed
Guest 4 is connected
Connection is closed


In [36]:
try:
    connection = connector.connect(user="root",password="root@123")
    print("Connection established")
    cursor = connection.cursor()
except errors.Error as e:
    print('Error No: ',e.errno)
    print('Error Msg: ',e.msg)

Connection established


In [39]:
#use database little_lemon_db
cursor.execute("""USE little_lemon_db;""")

In [40]:
#Create a report containing the following information:
#The name and EmployeeID of the Little Lemon manager.
cursor.execute("""select employees.EmployeeID,employees.Name,employees.Role from employees where lower(role) = 'manager';""")
result = cursor.fetchall()
print(cursor.column_names)
for data in result:
    print(data)


('EmployeeID', 'Name', 'Role')
(1, 'Mario Gollini', 'Manager')
('Name', 'Annual_Salary')
('Mario Gollini', '$70,000')
('total_bookings',)
(Decimal('6'),)
('name', 'BookingID')
('Anees Java', 9)
('Bald Vin', 10)
('Jay Kon', 11)


In [41]:
#The name and role of the employee who receives the highest salary.
cursor.execute("""select employees.Name,employees.Annual_Salary from employees where employees.Annual_Salary = ALL (select max(employees.Annual_Salary) from employees);""")
result = cursor.fetchall()
print(cursor.column_names)
for data in result:
    print(data)
print()

('Name', 'Annual_Salary')
('Mario Gollini', '$70,000')



In [42]:
#The number of guests booked between 18:00 and 20:00.
cursor.execute("""with
	group_hour as (select Hour(bookings.BookingSlot) as hour,count(hour(bookings.BookingSlot)) as total_bookings from bookings where hour(bookings.BookingSlot) between '18' and '19' group by hour(bookings.BookingSlot))
    select sum(total_bookings) as total_bookings from group_hour;""")
result = cursor.fetchall()
print(cursor.column_names)
for data in result:
    print(data)
print()

('total_bookings',)
(Decimal('6'),)



In [44]:
#The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
cursor.execute("""select concat(bookings.GuestFirstName,' ',bookings.GuestLastName) as name,bookings.BookingID from bookings inner join employees on bookings.EmployeeID = employees.EmployeeID where lower(employees.Role) = 'receptionist';""")
result = cursor.fetchall()
print(cursor.column_names)
for data in result:
    print(data)

('name', 'BookingID')
('Anees Java', 9)
('Bald Vin', 10)
('Jay Kon', 11)


In [61]:
#Create a stored procedure named BasicSalesReport that returns the following statistics: 
#Total sales
#Average sale
#Minimum bill paid
#Maximum bill paid


#drop procedure if exists
cursor.execute("""DROP PROCEDURE IF EXISTS BasicSalesReport;""")

basic_sales_report = """
create procedure BasicSalesReport(out total_sales decimal(6,2),out average_sales decimal(6,2),out minimum_bill_paid decimal(6,2),out maximum_bill_paid decimal(6,2))
begin
select sum(orders.BillAmount) into total_sales from orders;
select avg(orders.BillAmount) into average_sales from orders;
select min(orders.BillAmount) into minimum_bill_paid from orders;
select max(orders.BillAmount) into maximum_bill_paid from orders;
end
"""
records=(0,1,2,3)
try:
    cursor.execute(basic_sales_report)
    result = cursor.callproc('BasicSalesReport',records)
    total_sales = result[0]
    average_sales = result[1]
    min_bill = result[2]
    max_bill = result[3]
    print(f"""
TOTAL SALE: {total_sales}
AVERAGE SALE: {average_sales}
MIN BILL: {min_bill}
MAX BILL: {max_bill}
 """)
except errors.Error as e:
    print(e)

# print(result.column_names)
# for data in dataset:
#     print(data)


TOTAL SALE: 243.00
AVERAGE SALE: 48.60
MIN BILL: 37.00
MAX BILL: 86.00
 


In [62]:
order_status ="""
select bookings.BookingSlot,concat(bookings.GuestFirstName,' ',bookings.GuestLastName) as name,employees.Name,employees.Role
from bookings
inner join employees
on bookings.EmployeeID = employees.EmployeeID
where lower(employees.Role) in ('assistant chef','head chef');
"""
try:
    cursor.execute(order_status)
    result = cursor.fetchall()
    for data in result:
        print(f"""
{data[0]}

{data[1]}

[Assigned to: {data[2]} {data[3]}]
""")
except errors.Error as e:
    print(e)


15:00:00

Vanessa McCarthy

[Assigned to: Giorgos Dioudis Head Chef]


17:30:00

Marcos Romero

[Assigned to: Fatma Kaya Assistant Chef]


20:00:00

Sam Jay

[Assigned to: Fatma Kaya Assistant Chef]

